# Classification by Neural Networks trial

In [1]:
# libraries

import pandas as pd
import numpy as np
import bz2
pd.set_option('display.max_columns', None)
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pingouin as pg
import statsmodels.api as sm
import matplotlib as mpl
from scipy import stats
from distfit import distfit
import scikit_posthocs as sp
import datetime as dt

pd.options.display.float_format = '{:.2f}'.format

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["image.cmap"] = "Pastel2"

import plotly.graph_objects as go
from math import isnan

# planting seed

import random

random.seed(10)

# importing data

df = pd.read_csv("/home/evida-monika/mhunters/final_data_4.csv.bz2", compression="bz2", sep=",")

# this is cleaned dataset
# we need to delete another user - ID = 235, because it has 274 sessions in 4 weeks - impossible.

df = df[df['id_users'] != 235]

# changing types of data

cols = ['name_en_exercises','discarded_session_execution', 'code_name_sessions', 
        'name_en_sessions', 'active_user_programs', 'completed_user_programs',
        'pro_programs', 'available_programs', 'name_en_programs', 
        'gender', 
        'activity_level', 'goal', 'body_type', 'newsletter_subscription', 'notifications_setting',
        'language', 'scientific_data_usage', 'BMI_category', 'name_en_implements']

for col in cols:
    df[col] = df[col].astype('category')
    
col_date = ['updated_at_ex_ex', 'created_at_exercises', 'updated_at_exercises',
            'updated_at_session_execution', 'created_at_user_programs', 'updated_at_user_programs',
            'created_at_programs', 'updated_at_programs', 'created_at_users', 'updated_at_users',
            'date_of_birth']

for col in col_date:
    df[col] = pd.to_datetime(df[col])#.dt.strftime("%Y-%m-%d %H:%M:%S")

cat_names = {
    'gender': {1: 'male', 0: 'female'},
    'activity_level': {0: 'very active', 1: 'active', 2: 'sedentary'},
    'goal': {0: 'lose', 1: 'gain', 2: 'antiaging'},
    'body_type': {0: 'thin', 1: 'mid', 2: 'strong'}
}

df = df.replace(cat_names)


# dataset only for users

df_users_only_once = df.drop_duplicates(subset=['id_users'], keep='first')

# here dropping all of the columns that are not relevant for users

df_users_only = df_users_only_once.loc[:, ['id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
                                          'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
                                          'total_calories_users2', 'kcal_per_session_users2', 
                                           'exercises_per_user', 'exercises_without_rest_per_user']]

# dataset for only one of id_session_execution

df_session_exe_only_once = df.drop_duplicates(subset=['id_session_execution'], keep='first')

# here dropping all of the columns that are not relevant for session_execution

df_session_exe = df_session_exe_only_once.loc[:, ['id_session_execution', 
                                                  'difficulty_feedback_session_execution',
       'enjoyment_feedback_session_execution',
       'reps_executed_session_execution', 'updated_at_session_execution',
       'discarded_session_execution', 'id_sessions', 'order_sessions',
       'time_duration_sessions', 'code_name_sessions', 'name_en_sessions',
       'id_user_programs', 'created_at_user_programs',
       'updated_at_user_programs', 'active_user_programs',
       'current_session_id_user_programs', 'completed_user_programs',
       'id_programs', 'created_at_programs', 'updated_at_programs',
       'pro_programs', 'available_programs', 'strength_programs',
       'endurance_programs', 'technique_programs', 'flexibility_programs',
       'intensity_programs', 'name_en_programs', 'description_en_programs',
       'id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
       'total_time_session_execution', 'total_reps_session_execution',
       'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
       'total_time_session_execution_min', 'reps_per_min_session_execution',
       'exercise_execution_time_min', 'calories_session_execution',
       'total_calories_users2', 'kcal_per_session_users2',
                                          'YYYY/WW',
                                                 'number_exercises_in_session',
       'number_exercises_without_rest_in_session']]

# Creating a variable where there are 1 when it is the first time that user did a session in a day and 0 if the user already did a session that day

df_session_exe_copy = df_session_exe.copy()

df_session_exe_copy = df_session_exe_copy.sort_values(['id_users', 'updated_at_session_execution'])

df_session_exe_copy['days_a_week_session_exe'] = np.where((df_session_exe_copy['YYYY/WW'] == df_session_exe_copy['YYYY/WW'].shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.year == df_session_exe_copy['updated_at_session_execution'].dt.year.shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.month == df_session_exe_copy['updated_at_session_execution'].dt.month.shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.day == df_session_exe_copy['updated_at_session_execution'].dt.day.shift()) & 
                                                (df_session_exe_copy['id_users'] == df_session_exe_copy['id_users'].shift()),
                                                0,1)


# grouped table with calculation of days trained a week for every user and training days setting by the user

df_session_exe_copy2 = df_session_exe_copy.loc[:, ['id_users', 'YYYY/WW', 'days_a_week_session_exe']]

df_session_exe_days_setting = df_session_exe_copy2.groupby(['id_users', 'YYYY/WW'])['days_a_week_session_exe'].sum().reset_index()


df_users_training_days_setting = df_users_only.loc[:, ['id_users', 'training_days_setting']]

df_session_exe_days_setting = df_session_exe_days_setting.merge(df_users_training_days_setting, how = 'left', 
                                                                left_on = 'id_users', right_on = 'id_users')


# new column that says if user achieved weekly training goal (1) or not (0)

df_session_exe_days_setting['training_days_setting'] = df_session_exe_days_setting['training_days_setting'].astype('int64')

df_session_exe_days_setting['training_days_goal_achieved'] = np.where(df_session_exe_days_setting['days_a_week_session_exe'] >= df_session_exe_days_setting['training_days_setting'], 1, 0)


/home/evida-monika/anaconda3/envs/mhunters/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
import keras
import tensorflow

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

2023-03-30 11:54:28.630758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Introduction

ANN

## Predicting activity level

In [3]:
X_demo = df_users_only.loc[:, ['gender', 'body_type', 'newsletter_subscription', 
                                             'height', 'weight', 'body_fat']]

l_enc = LabelEncoder()

for col in ['gender', 'body_type', 'newsletter_subscription']:
    X_demo[col] = l_enc.fit_transform(X_demo[col]) # 1 - female, 0 - male, 0 - thin, 1 - mid, 2 - strong, 0 - True, 1 - False
    X_demo[col] = to_categorical(X_demo[col])

    

y_demo = []

for act in df_users_only['activity_level']:
    if act == 'active':
        y_demo.append(0)
    elif act == 'very active':
        y_demo.append(1)
    elif act == 'sedentary':
        y_demo.append(2)

y_demo = to_categorical(y_demo)
y_demo = np.array(y_demo)

y_dem = pd.DataFrame(y_demo, columns = ['activity_level_active', 'activity_level_very_active', 'activity_level_sedentary'])


In [4]:
sc = StandardScaler()
X_demo = pd.DataFrame(sc.fit_transform(X_demo))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_demo, y_demo, test_size = 0.3, random_state = 42)

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2123, 6), (2123, 3), (911, 6), (911, 3))

In [7]:
ann = Sequential()

2023-03-28 14:50:59.093919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
ann.add(Dense(units = 12, input_dim = len(X_train.columns), activation = 'relu'))
ann.add(Dense(units = 12, activation = 'relu'))
ann.add(Dense(units = 3, activation = 'softmax'))
ann.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                84        
                                                                 
 dense_1 (Dense)             (None, 12)                156       
                                                                 
 dense_2 (Dense)             (None, 3)                 39        
                                                                 
Total params: 279
Trainable params: 279
Non-trainable params: 0
_________________________________________________________________


In [9]:
ann.fit(X_train, y_train, batch_size = 15, epochs = 20, verbose = 1)

Epoch 1/20
142/142 [==============================] - 1s 939us/step - loss: 0.9866 - accuracy: 0.5280
Epoch 2/20
142/142 [==============================] - 0s 911us/step - loss: 0.8814 - accuracy: 0.5977
Epoch 3/20
142/142 [==============================] - 0s 922us/step - loss: 0.8596 - accuracy: 0.6015
Epoch 4/20
142/142 [==============================] - 0s 970us/step - loss: 0.8509 - accuracy: 0.6043
Epoch 5/20
142/142 [==============================] - 0s 931us/step - loss: 0.8468 - accuracy: 0.6086
Epoch 6/20
142/142 [==============================] - 0s 875us/step - loss: 0.8437 - accuracy: 0.6062
Epoch 7/20
142/142 [==============================] - 0s 927us/step - loss: 0.8403 - accuracy: 0.6076
Epoch 8/20
142/142 [==============================] - 0s 828us/step - loss: 0.8388 - accuracy: 0.6109
Epoch 9/20
142/142 [==============================] - 0s 835us/step - loss: 0.8371 - accuracy: 0.6067
Epoch 10/20
142/142 [==============================] - 0s 811us/step - loss: 0.834

In [10]:
ann.evaluate(X_test, y_test, verbose = 1, batch_size = 15)

61/61 [==============================] - 0s 780us/step - loss: 0.8520 - accuracy: 0.5807


[0.8520185351371765, 0.5806805491447449]

In [11]:
y_pred_class = (ann.predict(X_test) > 0.5).astype('int32')

29/29 [==============================] - 0s 675us/step


In [12]:
y_pred_class

array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=int32)

In [13]:
y_pred_class.shape

(911, 3)

In [14]:
y_test = y_test.astype('int32')

In [15]:
y_test = pd.DataFrame(y_test)

In [16]:
y_dem['activity_level'] = np.where(y_dem['activity_level_active'] == 1,
                                   0,
                                   np.where(y_dem['activity_level_very_active'] == 1, 
                                            1, 
                                            2))

In [17]:
y_dem = y_dem.loc[:, ['activity_level']]

In [18]:
y_test['observed'] = np.where(y_test[0] == 1,
                                   0,
                                   np.where(y_test[1] == 1, 
                                            1, 
                                            2))

y_test = y_test.loc[:, ['observed']]

In [19]:
y_pred_class = pd.DataFrame(y_pred_class)

In [20]:
y_pred_class['predicted'] = np.where(y_pred_class[0] == 1,
                                   0,
                                   np.where(y_pred_class[1] == 1, 
                                            1, 
                                            2))

y_pred_class = y_pred_class.loc[:, ['predicted']]

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
cm = confusion_matrix(y_test, y_pred_class, labels = [0,1,2])

In [23]:
cm

array([[426,   0,  67],
       [ 92,   0,   6],
       [205,   0, 115]])

In [24]:
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.59      0.86      0.70       493
           1       0.00      0.00      0.00        98
           2       0.61      0.36      0.45       320

    accuracy                           0.59       911
   macro avg       0.40      0.41      0.38       911
weighted avg       0.53      0.59      0.54       911



In [25]:
n_samples = len(y_test)

overall_accuracy = np.sum(y_pred_class['predicted'] == y_test['observed']) / n_samples

In [26]:
overall_accuracy

0.5938529088913282

In [43]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

# (https://datagraphi.com/blog/post/2019/12/17/how-to-find-the-optimum-number-of-hidden-layers-and-nodes-in-a-neural-network-model)

In [45]:
def make_classification_ann(optimizer, act, n_class, n_layers, first_layer_nodes, metrics):
    mod = Sequential()
    
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, n_class)
    for i in range(1, n_layers):
        if i==1:
            mod.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act))
        else:
            mod.add(Dense(n_nodes[i-1], activation=act))
    if n_class != 2:
        mod.add(Dense(units=n_class, activation='softmax'))
        mod.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[metrics])
    
    else: 
        mod.add(Dense(units=n_class, activation='sigmoid'))
        mod.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])

    return mod


In [28]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [46]:
classifier_for_search = KerasClassifier(make_classification_ann, verbose=1)


In [59]:
parameters = {'batch_size':[10,15,20],
              'epochs':[10,15,20],
              'optimizer':['adam'],
              'n_class': [y_demo.shape[1]],
              'act': ['relu'],
              'n_layers': [2,3],
              'first_layer_nodes': [10,8,6,4],
              'metrics': ['accuracy']
                 }

In [57]:
grid_search=GridSearchCV(estimator=classifier_for_search, param_grid=parameters, scoring='f1', cv=5)

In [52]:
grid_search.fit(X_train,y_train, verbose=1)

Epoch 1/10
170/170 [==============================] - 1s 835us/step - loss: 1.0393 - accuracy: 0.4187
Epoch 2/10
170/170 [==============================] - 0s 891us/step - loss: 0.9166 - accuracy: 0.5400
Epoch 3/10
170/170 [==============================] - 0s 870us/step - loss: 0.8886 - accuracy: 0.5518
Epoch 4/10
170/170 [==============================] - 0s 882us/step - loss: 0.8753 - accuracy: 0.5512
Epoch 5/10
170/170 [==============================] - 0s 868us/step - loss: 0.8678 - accuracy: 0.5748
Epoch 6/10
170/170 [==============================] - 0s 836us/step - loss: 0.8633 - accuracy: 0.5931
Epoch 7/10
170/170 [==============================] - 0s 854us/step - loss: 0.8605 - accuracy: 0.5931
Epoch 8/10
170/170 [==============================] - 0s 869us/step - loss: 0.8579 - accuracy: 0.5913
Epoch 9/10
170/170 [==============================] - 0s 833us/step - loss: 0.8559 - accuracy: 0.5948
Epoch 10/10
14/14 [==============================] - 0s 678us/step
Epoch 1/10
170/

170/170 [==============================] - 0s 819us/step - loss: 0.8968 - accuracy: 0.5771
Epoch 4/10
170/170 [==============================] - 0s 842us/step - loss: 0.8741 - accuracy: 0.5913
Epoch 5/10
170/170 [==============================] - 0s 835us/step - loss: 0.8627 - accuracy: 0.5907
Epoch 6/10
170/170 [==============================] - 0s 830us/step - loss: 0.8557 - accuracy: 0.5960
Epoch 7/10
170/170 [==============================] - 0s 837us/step - loss: 0.8507 - accuracy: 0.5913
Epoch 8/10
170/170 [==============================] - 0s 809us/step - loss: 0.8482 - accuracy: 0.5984
Epoch 9/10
170/170 [==============================] - 0s 819us/step - loss: 0.8459 - accuracy: 0.5995
Epoch 10/10
14/14 [==============================] - 0s 695us/step
Epoch 1/10
170/170 [==============================] - 0s 839us/step - loss: 1.1062 - accuracy: 0.3592
Epoch 2/10
170/170 [==============================] - 0s 821us/step - loss: 0.9611 - accuracy: 0.5442
Epoch 3/10
170/170 [======

170/170 [==============================] - 0s 871us/step - loss: 0.8988 - accuracy: 0.5654
Epoch 6/10
170/170 [==============================] - 0s 878us/step - loss: 0.8921 - accuracy: 0.5754
Epoch 7/10
170/170 [==============================] - 0s 838us/step - loss: 0.8879 - accuracy: 0.5807
Epoch 8/10
170/170 [==============================] - 0s 850us/step - loss: 0.8843 - accuracy: 0.5842
Epoch 9/10
170/170 [==============================] - 0s 855us/step - loss: 0.8801 - accuracy: 0.5913
Epoch 10/10
14/14 [==============================] - 0s 654us/step
Epoch 1/10
170/170 [==============================] - 0s 854us/step - loss: 1.3268 - accuracy: 0.4276
Epoch 2/10
170/170 [==============================] - 0s 861us/step - loss: 1.0647 - accuracy: 0.4688
Epoch 3/10
170/170 [==============================] - 0s 884us/step - loss: 0.9637 - accuracy: 0.5395
Epoch 4/10
170/170 [==============================] - 0s 879us/step - loss: 0.9155 - accuracy: 0.5766
Epoch 5/10
170/170 [======

170/170 [==============================] - 0s 817us/step - loss: 0.9654 - accuracy: 0.5680
Epoch 2/10
170/170 [==============================] - 0s 827us/step - loss: 0.9022 - accuracy: 0.5898
Epoch 3/10
170/170 [==============================] - 0s 832us/step - loss: 0.8772 - accuracy: 0.5886
Epoch 4/10
170/170 [==============================] - 0s 853us/step - loss: 0.8645 - accuracy: 0.5956
Epoch 5/10
170/170 [==============================] - 0s 877us/step - loss: 0.8576 - accuracy: 0.5951
Epoch 6/10
170/170 [==============================] - 0s 885us/step - loss: 0.8528 - accuracy: 0.5962
Epoch 7/10
170/170 [==============================] - 0s 870us/step - loss: 0.8491 - accuracy: 0.6009
Epoch 8/10
170/170 [==============================] - 0s 843us/step - loss: 0.8468 - accuracy: 0.6015
Epoch 9/10
170/170 [==============================] - 0s 851us/step - loss: 0.8449 - accuracy: 0.6021
Epoch 10/10
14/14 [==============================] - 0s 650us/step
Epoch 1/10
170/170 [======

14/14 [==============================] - 0s 671us/step
Epoch 1/15
170/170 [==============================] - 0s 812us/step - loss: 1.1263 - accuracy: 0.3685
Epoch 2/15
170/170 [==============================] - 0s 847us/step - loss: 0.9281 - accuracy: 0.5491
Epoch 3/15
170/170 [==============================] - 0s 821us/step - loss: 0.8784 - accuracy: 0.5762
Epoch 4/15
170/170 [==============================] - 0s 819us/step - loss: 0.8620 - accuracy: 0.5974
Epoch 5/15
170/170 [==============================] - 0s 840us/step - loss: 0.8555 - accuracy: 0.6074
Epoch 6/15
170/170 [==============================] - 0s 825us/step - loss: 0.8518 - accuracy: 0.6068
Epoch 7/15
170/170 [==============================] - 0s 824us/step - loss: 0.8492 - accuracy: 0.6057
Epoch 8/15
170/170 [==============================] - 0s 832us/step - loss: 0.8474 - accuracy: 0.6109
Epoch 9/15
170/170 [==============================] - 0s 862us/step - loss: 0.8456 - accuracy: 0.6045
Epoch 10/15
170/170 [======

...

85/85 [==============================] - 0s 893us/step - loss: 0.8265 - accuracy: 0.6121
Epoch 19/20
85/85 [==============================] - 0s 879us/step - loss: 0.8257 - accuracy: 0.6109
Epoch 20/20
14/14 [==============================] - 0s 836us/step
Epoch 1/20
85/85 [==============================] - 0s 885us/step - loss: 1.1353 - accuracy: 0.3896
Epoch 2/20
85/85 [==============================] - 0s 837us/step - loss: 1.0181 - accuracy: 0.4391
Epoch 3/20
85/85 [==============================] - 0s 812us/step - loss: 0.9450 - accuracy: 0.5150
Epoch 4/20
85/85 [==============================] - 0s 864us/step - loss: 0.8998 - accuracy: 0.5880
Epoch 5/20
85/85 [==============================] - 0s 916us/step - loss: 0.8762 - accuracy: 0.6015
Epoch 6/20
85/85 [==============================] - 0s 859us/step - loss: 0.8622 - accuracy: 0.6051
Epoch 7/20
85/85 [==============================] - 0s 881us/step - loss: 0.8559 - accuracy: 0.6057
Epoch 8/20
85/85 [=========================

85/85 [==============================] - 0s 844us/step - loss: 0.8420 - accuracy: 0.6078
Epoch 17/20
85/85 [==============================] - 0s 864us/step - loss: 0.8416 - accuracy: 0.6060
Epoch 18/20
85/85 [==============================] - 0s 799us/step - loss: 0.8414 - accuracy: 0.6078
Epoch 19/20
85/85 [==============================] - 0s 872us/step - loss: 0.8410 - accuracy: 0.6037
Epoch 20/20
14/14 [==============================] - 0s 633us/step
Epoch 1/20
85/85 [==============================] - 0s 851us/step - loss: 1.1546 - accuracy: 0.3068
Epoch 2/20
85/85 [==============================] - 0s 852us/step - loss: 1.0238 - accuracy: 0.4594
Epoch 3/20
85/85 [==============================] - 0s 824us/step - loss: 0.9548 - accuracy: 0.5636
Epoch 4/20
85/85 [==============================] - 0s 837us/step - loss: 0.9143 - accuracy: 0.5936
Epoch 5/20
85/85 [==============================] - 0s 834us/step - loss: 0.8880 - accuracy: 0.6090
Epoch 6/20
85/85 [=======================

85/85 [==============================] - 0s 772us/step - loss: 0.8512 - accuracy: 0.5980
Epoch 15/20
85/85 [==============================] - 0s 866us/step - loss: 0.8501 - accuracy: 0.6021
Epoch 16/20
85/85 [==============================] - 0s 803us/step - loss: 0.8489 - accuracy: 0.5974
Epoch 17/20
85/85 [==============================] - 0s 848us/step - loss: 0.8483 - accuracy: 0.5968
Epoch 18/20
85/85 [==============================] - 0s 795us/step - loss: 0.8476 - accuracy: 0.5974
Epoch 19/20
85/85 [==============================] - 0s 780us/step - loss: 0.8471 - accuracy: 0.5968
Epoch 20/20
14/14 [==============================] - 0s 879us/step
Epoch 1/20
85/85 [==============================] - 0s 883us/step - loss: 1.1674 - accuracy: 0.2992
Epoch 2/20
85/85 [==============================] - 0s 861us/step - loss: 1.0548 - accuracy: 0.4122
Epoch 3/20
85/85 [==============================] - 0s 877us/step - loss: 0.9983 - accuracy: 0.5400
Epoch 4/20
85/85 [=====================

85/85 [==============================] - 0s 929us/step - loss: 0.8380 - accuracy: 0.6184
Epoch 13/20
85/85 [==============================] - 0s 882us/step - loss: 0.8370 - accuracy: 0.6172
Epoch 14/20
85/85 [==============================] - 0s 878us/step - loss: 0.8362 - accuracy: 0.6160
Epoch 15/20
85/85 [==============================] - 0s 888us/step - loss: 0.8355 - accuracy: 0.6160
Epoch 16/20
85/85 [==============================] - 0s 896us/step - loss: 0.8349 - accuracy: 0.6143
Epoch 17/20
85/85 [==============================] - 0s 833us/step - loss: 0.8344 - accuracy: 0.6160
Epoch 18/20
85/85 [==============================] - 0s 793us/step - loss: 0.8340 - accuracy: 0.6166
Epoch 19/20
85/85 [==============================] - 0s 913us/step - loss: 0.8337 - accuracy: 0.6172
Epoch 20/20
14/14 [==============================] - 0s 699us/step
Epoch 1/20
85/85 [==============================] - 0s 820us/step - loss: 1.3790 - accuracy: 0.3531
Epoch 2/20
85/85 [===================

ZeroDivisionError: division by zero

In [87]:
grid_search.best_params_

{'act': 'relu',
 'batch_size': 10,
 'epochs': 10,
 'n_class': 3,
 'neurons': 12,
 'optimizer': 'adam'}